In [36]:
import os
import openai
from gitdb.util import write
from sympy import content
!pip install gitpython


In [37]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [38]:
from git import Repo

In [39]:
from pathlib import Path

In [40]:
pwd

'H:\\02_AIWork\\GitHub\\Data'

In [41]:
PATH_TO_BLOG_REPO = Path("H:\\02_AIWork\\GitHub\\Data\\.git")

In [42]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [43]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [44]:
PATH_TO_CONTENT

WindowsPath('H:/02_AIWork/GitHub/Data/content')

In [45]:
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [46]:
def update_blog(commit_message='Updates blog'):
    # GitPython -- Repo Location
    repo = Repo(PATH_TO_BLOG_REPO)
    # git add
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [47]:
random_text_string = "jakdfdjfkljasfljdjflsdajf"

In [48]:
with open(PATH_TO_BLOG/"index.html", "w") as f:
    f.write(random_text_string)

In [49]:
update_blog()

In [50]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)
    files =len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WHITE A NEW HTML FILE
        with open(path_to_new_content, "w") as f:
            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<meta charset="utf-8">\n')
            f.write('<title>' + title + '</title>\n')
            f.write('</head>\n')
            
            f.write('<body>\n')
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>\n")
            # OpenAI --> Completion GPT --> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write('</body>\n')
            f.write('</html>\n')
            print("Blog Created")
            return path_to_new_content
            
    else:
        raise FileNotFoundError("File already exists, please check again you name! Aborting!")
    

In [51]:
path_to_new_content = create_new_blog('Test_title','skdajfldjslkjdsf','example_download.png')

Blog Created


In [52]:
#index.html ---> Blog post

In [53]:
from bs4 import BeautifulSoup as Soup

In [54]:
# pip install BeautifulSoup4

In [57]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [58]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<title>Document</title>\n</head>\n<body>\n<h1>My blog Home page</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [ ]:
# checking duplicate links

# write blog post link ---> index.html

In [60]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(links.get("href")) for links in links] #[1.html,2.html,3..html]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [61]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all("a")    
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link already exists")
    links_to_new_blog = soup.new_tag("a",href=Path(*path_to_new_content.parts[-2:]))
    links_to_new_blog.string = path_to_new_content.name.split('.')[0]
    last_link.insert_after(links_to_new_blog)
    
    with open(PATH_TO_BLOG/'index.html', 'w') as f:
        f.write(str(soup.prettify(formatter="html")))

In [62]:
write_to_index(path_to_new_content)

In [ ]:
update_blog()